In [2]:
import os
import openai
import dotenv
from tqdm import tqdm
from datasets import load_dataset

dotenv.load_dotenv()

True

In [4]:
client = openai.OpenAI()

my_system = {
    "role": "system",
    "content": """
You are skilled programmer.
Currently you are working on on-call. 

So if you monitor the system, you can see that the system is running smoothly.
But if you got suspicious signal from the system, you call proper team to fix the issue.

If example like this, you call DB team to fix the issue.
Server[CPU]     : 10%
Server[Memory]  : 20%
Server[Disk]    : 5%
Server[Network] : 10%
DB[CPU]         : 95%
DB[Memory]      : 10%
DB[Disk]        : 5%
DB[Network]     : 10%
LB[CPU]         : 5%
LB[Memory]      : 10%
LB[Disk]        : 5%
LB[Network]     : 30%

If there is no suspicious signal, give no alert.

Monitoring system collect CPU, Memory, Disk, Network and 3 system Server, DB, LB.

Think step by step.
Must give reason why you call the team.
""",
}

# give me prompt CPU
ai_system = {
    "role": "system",
    "content": """
You are a skilled programmer on-call, monitoring the system to ensure it runs smoothly. Regularly check the system metrics for CPU, Memory, Disk, and Network usage across the Server, DB, and LB. If you detect any suspicious signals, such as CPU, Memory, Disk, or Network usage exceeding predefined thresholds, call the appropriate team to fix the issue. 

For example, if DB CPU usage exceeds 80%, notify the DB team.

Here is the current system status:
- Server[CPU]: 10%
- Server[Memory]: 20%
- Server[Disk]: 5%
- Server[Network]: 10%
- DB[CPU]: 95%
- DB[Memory]: 10%
- DB[Disk]: 5%
- DB[Network]: 10%
- LB[CPU]: 5%
- LB[Memory]: 10%
- LB[Disk]: 5%
- LB[Network]: 30%

Check each metric and take the following steps:
1. Identify any metrics that exceed the threshold.
2. Call the relevant team to address the issue.
3. Provide a clear reason for the alert.

In this case, you would call the DB team because the DB CPU usage is at 95%, which exceeds the 80% threshold.

""",
}

In [7]:
lb_my = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        my_system,
        {
            "role": "user",
            "content": """
Server[CPU]     : 10%
Server[Memory]  : 20%
Server[Disk]    : 5%
Server[Network] : 10%
DB[CPU]         : 10%
DB[Memory]      : 10%
DB[Disk]        : 5%
DB[Network]     : 10%
LB[CPU]         : 5%
LB[Memory]      : 10%
LB[Disk]        : 5%
LB[Network]     : 99%
""",
        },
    ],
)

no_my = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        my_system,
        {
            "role": "user",
            "content": """
Server[CPU]     : 10%
Server[Memory]  : 20%
Server[Disk]    : 5%
Server[Network] : 10%
DB[CPU]         : 10%
DB[Memory]      : 10%
DB[Disk]        : 5%
DB[Network]     : 10%
LB[CPU]         : 5%
LB[Memory]      : 10%
LB[Disk]        : 5%
LB[Network]     : 9%
""",
        },
    ],
)
print(lb_my.choices[0].message.content)
print(no_my.choices[0].message.content)

In this case, I observe the following metrics:

- **Server:**
  - CPU: 10%
  - Memory: 20%
  - Disk: 5%
  - Network: 10%

- **DB:**
  - CPU: 10%
  - Memory: 10%
  - Disk: 5%
  - Network: 10%

- **Load Balancer (LB):**
  - CPU: 5%
  - Memory: 10%
  - Disk: 5%
  - Network: 99%

**Analysis:**

1. **Server**: All metrics are well within the normal operating range.
2. **DB**: All metrics are normal, with CPU, Memory, Disk, and Network showing low utilization.
3. **Load Balancer**: The Network usage is at an alarming 99%. 

**Reason for Alert:**
The Load Balancer (LB) is experiencing extremely high network usage, which could lead to performance degradation or potential service interruption. High network utilization can result in slow response times or even timeouts if the Load Balancer cannot handle incoming traffic effectively. 

**Action:**
I would call the Load Balancer team to investigate the high network utilization and determine if there's an underlying issue (e.g., excessive traffic, 

In [9]:
lb_ai = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        ai_system,
        {
            "role": "user",
            "content": """
Server[CPU]     : 10%
Server[Memory]  : 20%
Server[Disk]    : 5%
Server[Network] : 10%
DB[CPU]         : 10%
DB[Memory]      : 10%
DB[Disk]        : 5%
DB[Network]     : 10%
LB[CPU]         : 5%
LB[Memory]      : 10%
LB[Disk]        : 5%
LB[Network]     : 99%
""",
        },
    ],
)

no_ai = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        ai_system,
        {
            "role": "user",
            "content": """
Server[CPU]     : 10%
Server[Memory]  : 20%
Server[Disk]    : 5%
Server[Network] : 10%
DB[CPU]         : 10%
DB[Memory]      : 10%
DB[Disk]        : 5%
DB[Network]     : 10%
LB[CPU]         : 5%
LB[Memory]      : 10%
LB[Disk]        : 5%
LB[Network]     : 9%
""",
        },
    ],
)
print(lb_ai.choices[0].message.content)
print(no_ai.choices[0].message.content)

1. **Identify any metrics that exceed the threshold:**
   - Server: No metrics exceed the threshold.
   - DB: No metrics exceed the threshold.
   - LB[Network]: 99% exceeds the threshold (typically 80%).

2. **Call the relevant team:**
   - Notify the Load Balancer (LB) team.

3. **Provide a clear reason for the alert:**
   - "Alert: Network usage of the Load Balancer (LB) has reached 99%, which exceeds the threshold of 80%. Immediate attention is required to prevent potential network bottlenecks or downtime."

Proceeding to call the LB team regarding the high network usage.
Based on the current system status, the metrics are as follows:

- Server[CPU]: 10%
- Server[Memory]: 20%
- Server[Disk]: 5%
- Server[Network]: 10%
- DB[CPU]: 10%
- DB[Memory]: 10%
- DB[Disk]: 5%
- DB[Network]: 10%
- LB[CPU]: 5%
- LB[Memory]: 10%
- LB[Disk]: 5%
- LB[Network]: 9%

Upon checking the system metrics, all values are within acceptable limits and do not exceed predefined thresholds. Therefore, there are n